In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark import SparkConf, SparkContext

import datetime

import numpy as np
import pandas as pd
import boto3
import configparser

import pyspark
import os

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1 pyspark-shell --master local[2] pyspark-shell --conf spark.hadoop.fs.s3a.endpoint=s3.us-west-2.amazonaws.com'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell --master local[2] pyspark-shell --conf spark.hadoop.fs.s3a.endpoint=s3.eu-west-3.amazonaws.com'
#org.apache.hadoop:hadoop-aws:2.7.4 before
%matplotlib inline
import matplotlib.pyplot as plt

import findspark
findspark.init()

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

s3 = boto3.resource('s3',
                       region_name="eu-west-3",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

In [3]:
#spark configuration
#conf = SparkConf().set("spark.executor.extraJavaOptions","-Dcom.amazonaws.services.s3.enableV4=true"). \
# set("spark.driver.extraJavaOptions","-Dcom.amazonaws.services.s3.enableV4=true"). \
# setAppName("data_wrangling").setMaster("local[*]")

#sc=SparkContext(conf=conf)
sc = pyspark.SparkContext(appName="data wrangling 2")
sc.setSystemProperty("com.amazonaws.services.s3.enableV4", "true")

accessKeyId=KEY
secretAccessKey=SECRET

hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3a.access.key", accessKeyId)
hadoopConf.set("fs.s3a.secret.key", secretAccessKey)
hadoopConf.set("fs.s3a.endpoint", "s3.eu-west-3.amazonaws.com")
hadoopConf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

spark=SparkSession(sc)

In [9]:
path = "s3a://pg-udacity2/songplays-small.csv"
#obj = s3.getObject("pg-udacity2","songplays.csv")
#print(obj.getObjectContent())

#bucket = s3.Bucket('pg-udacity2')
# Iterates through all the objects, doing the pagination for you. Each obj
# is an ObjectSummary, so it doesn't contain the body. You'll need to call
# get to get the whole body.
#for obj in bucket.objects.all():
#    key = obj.key
#    body = obj.get()['Body'].read()
#    print (body)

#user_log=spark.read.json(path)
user_log=spark.read.csv(path,header=False,inferSchema=True)
user_log.show(10)

+---------------+
|            _c0|
+---------------+
|    Deep Dreams|
|Data House Rock|
|    Deep Dreams|
|Data House Rock|
|Broken Networks|
|Data House Rock|
+---------------+

